# Optimal Choice: 
`L = [64, 64]`

`K = [64]`

test: the test set from original data
complete_test: the test set from completed_test data

In [1]:
import pandas as pd
import os
import json
import numpy as np


def collect_data_p(p: str, datasets=[], result_dir='results', train_test_flag="test"):
    """
    Collects data from multiple datasets and methods for a specific missingness percentage,
    and formats it into a DataFrame.
    
    Args:
        p (str): The missingness percentage (e.g., "0.0").
        datasets (list): List of dataset names to analyze.
        output_dir (str): The base directory where results are saved.
        
    Returns:
        pd.DataFrame: A DataFrame containing formatted results for each method and imputation.
    """
    if not datasets:
        datasets = [d for d in os.listdir(result_dir) if os.path.isdir(os.path.join(result_dir, d))]
    
    for dataset in datasets[1:]:
        dataset_dir = os.path.join(result_dir, dataset)
        method_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        break
    # print(method_dirs)
    method_dirs = [e for e in method_dirs if 'Balu' in e]

    # 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001'

    print("methods:", method_dirs)

    def fileter_method(must_item: list, method):
        method_name = method
        for item in must_item:
            if item not in method:
                return False, None
            method_name = method_name.replace("_"+item, "")
        return True, method_name

    data_rows = []
    for method in method_dirs:
        flag, method_name = fileter_method(['gconv=GCN_rconv=GCN', "reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001"], method)
        if not flag:
            continue

        row = {
            'method': method_name,
            # 'imputation': imputation
        }
        
        # Collect results for each dataset
        for dataset in datasets:
            dataset_dir = os.path.join(result_dir, dataset)
            if not os.path.exists(dataset_dir):
                print(f"{dataset_dir} not exits!")
                # Dataset doesn't exist, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
                
            method_dir = os.path.join(dataset_dir, method)
            if not os.path.exists(method_dir):
                # Method doesn't exist for this dataset, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
            
            imputation_dir = method_dir
            
            results_files = [f for f in os.listdir(imputation_dir) 
                        if f.startswith(f"p={p}_") and f.endswith(f"{train_test_flag}_results.json")]
            if "complete" not in train_test_flag:
                results_files = [f for f in results_files if "complete" not in f]

            # results_files=results_files[:len(results_files)*1.5//2]

            # print(results_files)
            effect_pehe_values = []
            effect_mae_values = []
            effect_sum_values = []
            
            for file in results_files:
                file_path = os.path.join(method_dir, file) # imputation, file)
                try:
                    with open(file_path, 'r') as f:
                        results = json.load(f)
                        if 'effect_pehe' in results:
                            effect_pehe_values.append(results['effect_pehe'])
                        if 'effect_mae' in results:
                            effect_mae_values.append(results['effect_mae'])

                        effect_sum_values.append(results['effect_pehe'] + results['effect_mae'])
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f"Warning: Error reading file {file_path}: {e}")
            
            if effect_pehe_values:
                mean_pehe = np.mean(effect_pehe_values)
                std_pehe = np.std(effect_pehe_values)
                row[f"{dataset}_PEHE"] = f"{mean_pehe:.2f} ± {std_pehe:.2f}"
            else:
                row[f"{dataset}_PEHE"] = "N/A"
            
            if effect_mae_values:
                mean_mae = np.mean(effect_mae_values)
                std_mae = np.std(effect_mae_values)
                row[f"{dataset}_MAE"] = f"{mean_mae:.2f} ± {std_mae:.2f}"
            else:
                row[f"{dataset}_MAE"] = "N/A"

            # if effect_sum_values:
            #     mean_mae = np.mean(effect_sum_values)
            #     std_mae = np.std(effect_sum_values)
            #     row[f"{dataset}_SUM"] = f"{method} {mean_mae:.2f} ± {std_mae:.2f}"
            # else:
            #     row[f"{dataset}_SUM"] = 'N/A'
        data_rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data_rows)
    # print(df)
    
    # Sort the DataFrame by method and imputation
    if not df.empty:
        df = df.sort_values(by=['method']) #, 'imputation'])
    return df

def display_results(result_dir, p, datasets=["Syn"], train_test_flag='test', metrics=['SUM']):
    

    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)  # Use the full width of the notebook cell
    pd.set_option('display.max_colwidth', None)  # Show full content of each column

    # Example: Display the DataFrame created by collect_data_p
    df = collect_data_p(p, datasets=datasets, result_dir=result_dir, train_test_flag=train_test_flag) #, "AMZS", "Flickr"])
    dataset_map = {'Syn_M=None_SimRel=1_Rel=4':"Instagram",  'BlogCatalog1_M=20_SimRel=0_Rel=1':"BlogCatalog", 'BlogCatalog1_M=20_SimRel=1_Rel=4':"BlogCatalog", 'Flickr1_M=20_SimRel=0_Rel=1':"Flickr", 'Flickr1_M=20_SimRel=1_Rel=4':"Flickr", 'Youtube_M=20_SimRel=1_Rel=4':"Youtube"}
    
    def replace_col(col):
        for k, v in dataset_map.items():
            if k in col:

                return col.replace(k, v).replace("_MCAR","")
        return col

    new_column_names = [replace_col(col) for col in df.columns]
    df.columns = new_column_names
    display(df)
    # print(df.to_latex(index=False, escape=False))
    # print(df.to_markdown(index=False))

# display_results(p='0.5', train_test_flag='test')

# Test with complete attributes

## MCAR

In [2]:
datasets = ["Syn_M=None", "BlogCatalog1_M=20", "Flickr_M=20", "Flickr1_M=20"]
datasets = ['BlogCatalog1_M=20_SimRel=1_Rel=4',
 'Flickr_M=20_SimRel=1_Rel=4',
 'Flickr1_M=20_SimRel=1_Rel=4',
 'AMZS_M=20_SimRel=1_Rel=4',
 'Youtube_M=20_SimRel=1_Rel=4']

datasets = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']     # , 'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4'
datasets = [e+"_MCAR" for e in datasets] 
result_dir = 'results_balu_tuning'

display_results(result_dir=result_dir, p='0.0', datasets=datasets, train_test_flag='test')
display_results(result_dir=result_dir, p='0.1', datasets=datasets, train_test_flag='test')
display_results(result_dir=result_dir, p='0.3', datasets=datasets, train_test_flag='test')
# display_results(result_dir='results_Q1', p='0.5', datasets=datasets, train_test_flag='test')


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
0,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64,0.79 ± 0.02,0.06 ± 0.04,0.79 ± 0.02,0.03 ± 0.01,6.68 ± 3.42,0.73 ± 0.41,7.58 ± 2.16,1.17 ± 0.45
1,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64,0.79 ± 0.02,0.06 ± 0.04,0.79 ± 0.02,0.03 ± 0.01,6.66 ± 3.43,0.67 ± 0.37,7.58 ± 2.15,1.18 ± 0.45
9,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64,0.80 ± 0.03,0.05 ± 0.04,0.79 ± 0.02,0.03 ± 0.03,7.30 ± 3.72,0.93 ± 0.47,7.25 ± 1.73,1.03 ± 0.46
6,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64-64,0.80 ± 0.03,0.06 ± 0.04,0.79 ± 0.02,0.03 ± 0.02,7.39 ± 3.90,0.98 ± 0.51,7.26 ± 1.73,1.01 ± 0.47
8,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64,0.78 ± 0.01,0.07 ± 0.03,0.79 ± 0.02,0.03 ± 0.03,7.78 ± 6.56,0.90 ± 1.02,7.52 ± 2.51,1.17 ± 0.55
3,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64-64,0.78 ± 0.01,0.07 ± 0.03,0.79 ± 0.02,0.03 ± 0.03,7.78 ± 6.56,0.90 ± 1.02,7.52 ± 2.51,1.17 ± 0.55
7,Balu_plus_imp=BaLu_IGMCL=64-64_K=64,0.80 ± 0.02,0.06 ± 0.03,0.79 ± 0.02,0.03 ± 0.02,7.45 ± 3.96,0.90 ± 0.61,7.44 ± 1.76,1.09 ± 0.38
5,Balu_plus_imp=BaLu_IGMCL=64-64_K=64-64,0.80 ± 0.02,0.06 ± 0.03,0.79 ± 0.02,0.03 ± 0.02,7.45 ± 3.96,0.90 ± 0.61,7.44 ± 1.76,1.09 ± 0.38
4,Balu_plus_imp=GRAPEL=64-64_K=64,0.79 ± 0.02,0.05 ± 0.03,0.81 ± 0.02,0.03 ± 0.03,12.13 ± 9.21,1.83 ± 1.25,15.19 ± 2.68,3.11 ± 2.40
2,Balu_plus_imp=IGMCL=64-64_K=64,0.79 ± 0.02,0.06 ± 0.03,0.79 ± 0.01,0.03 ± 0.02,11.97 ± 9.53,1.89 ± 1.20,15.06 ± 2.50,3.14 ± 2.31


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
0,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64,1.87 ± 0.21,0.32 ± 0.22,1.91 ± 0.21,0.13 ± 0.05,9.68 ± 9.45,0.97 ± 0.66,8.08 ± 1.98,1.12 ± 0.63
1,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64,1.87 ± 0.21,0.32 ± 0.22,1.91 ± 0.21,0.13 ± 0.05,9.65 ± 9.46,0.94 ± 0.65,8.08 ± 1.98,1.12 ± 0.62
9,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64,1.88 ± 0.23,0.31 ± 0.19,1.91 ± 0.20,0.12 ± 0.06,7.64 ± 4.26,0.92 ± 0.52,7.94 ± 2.85,0.59 ± 0.38
6,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64-64,1.88 ± 0.23,0.31 ± 0.19,1.91 ± 0.20,0.12 ± 0.06,7.63 ± 4.26,0.88 ± 0.55,7.94 ± 2.85,0.59 ± 0.38
8,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64,1.88 ± 0.22,0.34 ± 0.24,1.92 ± 0.22,0.10 ± 0.06,8.61 ± 6.49,0.74 ± 0.72,6.78 ± 0.93,0.70 ± 0.26
3,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64-64,1.88 ± 0.22,0.34 ± 0.24,1.92 ± 0.22,0.10 ± 0.06,8.61 ± 6.49,0.74 ± 0.72,6.78 ± 0.93,0.70 ± 0.26
7,Balu_plus_imp=BaLu_IGMCL=64-64_K=64,1.85 ± 0.20,0.29 ± 0.19,1.91 ± 0.21,0.10 ± 0.08,8.16 ± 4.28,0.62 ± 0.33,8.00 ± 1.79,0.87 ± 0.82
5,Balu_plus_imp=BaLu_IGMCL=64-64_K=64-64,1.85 ± 0.20,0.29 ± 0.19,1.91 ± 0.21,0.10 ± 0.08,8.16 ± 4.28,0.62 ± 0.33,8.00 ± 1.79,0.87 ± 0.82
4,Balu_plus_imp=GRAPEL=64-64_K=64,1.87 ± 0.22,0.35 ± 0.23,1.91 ± 0.20,0.11 ± 0.07,9.86 ± 2.80,1.80 ± 1.36,15.20 ± 2.36,2.65 ± 1.83
2,Balu_plus_imp=IGMCL=64-64_K=64,1.86 ± 0.23,0.34 ± 0.22,1.91 ± 0.21,0.13 ± 0.06,9.40 ± 3.10,1.80 ± 1.18,15.24 ± 2.47,2.91 ± 2.13


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
0,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64,3.14 ± 0.62,1.08 ± 0.80,3.13 ± 0.38,0.27 ± 0.14,7.44 ± 3.89,1.31 ± 0.60,7.51 ± 1.08,0.72 ± 0.44
1,Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64,3.13 ± 0.60,1.07 ± 0.78,3.13 ± 0.38,0.27 ± 0.14,7.42 ± 3.90,1.31 ± 0.58,7.51 ± 1.08,0.70 ± 0.46
9,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64,3.14 ± 0.60,1.09 ± 0.78,3.13 ± 0.36,0.23 ± 0.13,7.65 ± 3.93,1.23 ± 0.50,7.24 ± 0.89,0.58 ± 0.45
6,Balu_plus_imp=BaLu_GRAPEL=64-64_K=64-64,3.14 ± 0.60,1.09 ± 0.78,3.13 ± 0.36,0.23 ± 0.13,7.65 ± 3.92,1.22 ± 0.52,7.24 ± 0.89,0.59 ± 0.44
8,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64,3.16 ± 0.62,1.02 ± 0.74,3.15 ± 0.39,0.20 ± 0.13,8.50 ± 4.21,1.05 ± 0.55,7.41 ± 0.94,0.92 ± 0.42
3,Balu_plus_imp=BaLu_IGMCL=64-64-64_K=64-64,3.16 ± 0.62,1.02 ± 0.74,3.15 ± 0.39,0.20 ± 0.13,8.50 ± 4.21,1.05 ± 0.55,7.41 ± 0.94,0.92 ± 0.42
7,Balu_plus_imp=BaLu_IGMCL=64-64_K=64,3.10 ± 0.60,1.08 ± 0.81,3.12 ± 0.34,0.22 ± 0.17,9.44 ± 5.64,1.65 ± 0.79,7.46 ± 1.14,0.73 ± 0.55
5,Balu_plus_imp=BaLu_IGMCL=64-64_K=64-64,3.10 ± 0.60,1.08 ± 0.81,3.12 ± 0.34,0.22 ± 0.17,9.44 ± 5.64,1.65 ± 0.79,7.46 ± 1.14,0.73 ± 0.55
4,Balu_plus_imp=GRAPEL=64-64_K=64,3.13 ± 0.56,1.07 ± 0.76,3.14 ± 0.38,0.30 ± 0.19,10.12 ± 3.03,2.42 ± 1.23,15.53 ± 2.66,2.58 ± 1.37
2,Balu_plus_imp=IGMCL=64-64_K=64,3.10 ± 0.56,1.06 ± 0.77,3.12 ± 0.36,0.26 ± 0.16,9.41 ± 2.59,2.09 ± 1.28,15.46 ± 2.64,2.40 ± 1.54
